<a href="https://colab.research.google.com/github/Wn013/-tool-Download-Upload-Google-Drive-version-1.5/blob/main/Copy_Folder_Google_Drive_to_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Copy Folder Google Drive to Google Drive - 1TouchPro

In [ ]:
#@title Script Tối ưu Hơn (Giảm Log & Sleep) - Chặn Sao chép/Tải xuống Đệ Quy

import time
import re
import sys
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# --- Biến toàn cục và Callback ---
batch_results = {'success': 0, 'fail': 0}

def batch_callback(request_id, response, exception):
    """Callback xử lý kết quả batch, giảm log thành công."""
    global batch_results
    file_name = request_id
    if exception:
        print(f"    [BATCH FAIL] Lỗi khi cập nhật '{file_name}': {exception}")
        batch_results['fail'] += 1
    else:
        # Chỉ tăng bộ đếm, không in ra nếu thành công để giảm log
        batch_results['success'] += 1
        # Có thể thêm kiểm tra response ở đây nếu muốn log Warning như trước

# --- Các hàm get_drive_service, extract_folder_id_from_url giữ nguyên ---
def get_drive_service():
    try:
        auth.authenticate_user()
        service = build('drive', 'v3', cache_discovery=False)
        print("Xác thực thành công và đã kết nối với Google Drive API.")
        return service
    except Exception as e:
        print(f"Lỗi khi xác thực hoặc tạo Drive service: {e}")
        sys.exit("Không thể xác thực, dừng script.")

def extract_folder_id_from_url(url):
    patterns = [
        r'/folders/([-\w]{25,})', r'/drive/folders/([-\w]{25,})',
        r'id=([-\w]{25,})', r'/file/d/([-\w]{25,})' ,
        r'/drive/u/[0-9]+/folders/([-\w]{25,})'
    ]
    for pattern in patterns:
         match = re.search(pattern, url)
         if match: return match.group(1)
    if '/' in url:
         potential_id = url.split('/')[-1].split('?')[0]
         if len(potential_id) >= 25 and '.' not in potential_id: return potential_id
    return None

# --- HÀM ĐỆ QUY ĐÃ TỐI ƯU LOG VÀ SLEEP ---
def process_folder_recursively_batch_optimized(service, folder_id, folder_name=""):
    """Xử lý đệ quy, giảm log và sleep."""
    print(f"Processing Folder: '{folder_name}' (ID: {folder_id})") # Giữ log vào/ra thư mục
    page_token = None
    batch = service.new_batch_http_request(callback=batch_callback)
    batch_count = 0
    BATCH_SIZE = 50 # Giữ nguyên batch size

    while True: # Pagination loop
        try:
            response = service.files().list(
                q=f"'{folder_id}' in parents and trashed = false",
                fields="nextPageToken, files(id, name, mimeType)",
                pageToken=page_token,
                supportsAllDrives=True,
                includeItemsFromAllDrives=True,
                pageSize=500
            ).execute()
            items = response.get('files', [])

            if not items and page_token is None: break

            for item in items:
                item_id = item.get('id')
                item_name = item.get('name', 'Không rõ tên')
                item_mime_type = item.get('mimeType')

                if not item_id: continue

                if item_mime_type == 'application/vnd.google-apps.folder':
                    # Thực thi batch hiện tại trước khi đệ quy
                    if batch_count > 0:
                        # print(f"  Executing batch of {batch_count} updates before subfolder...") # Giảm log
                        try: batch.execute()
                        except HttpError as batch_error: print(f"  [BATCH EXECUTE ERROR]: {batch_error}")
                        time.sleep(0.2) # Giảm sleep sau batch execute
                        batch = service.new_batch_http_request(callback=batch_callback)
                        batch_count = 0
                    # Gọi đệ quy
                    process_folder_recursively_batch_optimized(service, item_id, item_name)
                else:
                    # Thêm file vào batch
                    update_body = {'copyRequiresWriterPermission': True}
                    request = service.files().update(
                        fileId=item_id, body=update_body, supportsAllDrives=True, fields='id' # Chỉ cần ID là đủ
                    )
                    batch.add(request, request_id=item_name)
                    batch_count += 1

                    if batch_count >= BATCH_SIZE:
                        # print(f"  Executing batch of {batch_count} file updates...") # Giảm log
                        try: batch.execute()
                        except HttpError as batch_error: print(f"  [BATCH EXECUTE ERROR]: {batch_error}")
                        time.sleep(0.2) # Giảm sleep sau batch execute
                        batch = service.new_batch_http_request(callback=batch_callback)
                        batch_count = 0

            page_token = response.get('nextPageToken', None)
            if page_token is None: break
            # time.sleep(0.1) # Giảm sleep giữa các trang

        except HttpError as error:
            print(f"  [API ERROR] Lỗi khi liệt kê mục trong '{folder_name}': {error}. Bỏ qua thư mục này.")
            break # Thoát xử lý thư mục này nếu không list được
        except Exception as e:
            print(f"  [OTHER ERROR] Lỗi khác khi xử lý '{folder_name}': {e}. Bỏ qua thư mục này.")
            break

    # Thực thi nốt batch cuối cùng
    if batch_count > 0:
        # print(f"  Executing final batch of {batch_count} updates for '{folder_name}'...") # Giảm log
        try: batch.execute()
        except HttpError as batch_error: print(f"  [BATCH EXECUTE ERROR] Lỗi batch cuối: {batch_error}")
        # Không cần sleep sau batch cuối cùng của một thư mục

    # print(f"Finished processing folder: '{folder_name}'.") # Giảm log

# --- Logic chính ---
if __name__ == "__main__":
    print("--- Script Tối ưu Hơn (Batch + Đệ Quy) Chặn Sao chép/Tải xuống ---")
    drive_service = get_drive_service()

    if drive_service:
        folder_link = input("Nhập link thư mục Google Drive GỐC bạn muốn xử lý (sẽ xử lý cả thư mục con): ")
        top_folder_id = extract_folder_id_from_url(folder_link)

        if top_folder_id:
            try:
                top_folder_info = drive_service.files().get(fileId=top_folder_id, fields='id, name', supportsAllDrives=True).execute()
                top_folder_name = top_folder_info.get('name', top_folder_id)
            except Exception: top_folder_name = top_folder_id

            print(f"\nBắt đầu xử lý đệ quy từ thư mục '{top_folder_name}' (ID: {top_folder_id})...")
            start_time = time.time()
            batch_results = {'success': 0, 'fail': 0} # Reset bộ đếm

            try:
                 process_folder_recursively_batch_optimized(drive_service, top_folder_id, top_folder_name)
            except Exception as e:
                 print(f"\n !!! LỖI KHÔNG XÁC ĐỊNH TRONG QUÁ TRÌNH ĐỆ QUY: {e} !!!")

            end_time = time.time()
            duration = end_time - start_time
            total_processed = batch_results['success'] + batch_results['fail']

            print("\n--- HOÀN THÀNH QUÁ TRÌNH ĐỆ QUY ---")
            print(f"Tổng thời gian thực thi: {duration:.2f} giây ({duration/60:.2f} phút)")
            print(f"Tổng số API updates đã được gửi (ước tính): {total_processed}")
            print(f"-> Thành công: {batch_results['success']}")
            print(f"-> Thất bại: {batch_results['fail']}")
            print("Lưu ý:")
            print("- Đã giảm bớt thông báo log và thời gian nghỉ để tăng tốc độ.")
            print("- NẾU GẶP LỖI 403 (Rate Limit) hoặc 429 thường xuyên, hãy cân nhắc tăng thời gian nghỉ (time.sleep) sau mỗi batch.execute() lên một chút (vd: 0.5 hoặc 1.0 giây).")
            if batch_results['fail'] > 0: print("- Vui lòng kiểm tra log lỗi bên trên.")
        else:
            print("Không thể lấy ID thư mục từ link bạn cung cấp.")